# How to Inspect the XSF File Content

This notebook shows how to load and display the content of an Extensible Sounder Format (XSF) File

## Import the Required Libraries

In [ ]:
# use 'matplotlib inline' to avoid the plot interactivity
%matplotlib notebook

import os
import numpy as np
import netCDF4  as nc
import matplotlib.pyplot as plt
from hyo2.abc.lib.testing import Testing

## Retrieve XSF File Path

In [ ]:
# index of the XSF to use (it is a parameter!)
xsf_idx = 0

# use an helper class to retrieve all the files with .xsf extension in the "download" folder
data_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
testing = Testing(root_folder=data_folder)
xsf_files = testing.download_test_files(ext=".xsf")
nr_xsf_files = len(xsf_files)
if xsf_idx >= nr_xsf_files:
    raise RuntimeError("Not enough XSF files were retrieved for using %d index!" % xsf_idx)
print("Nr. of available XSF files: %s" % nr_xsf_files)

# select the XSF file path
xsf_path = xsf_files[xsf_idx]
print("Path to the selected XSF file: %s" % xsf_path)

## Plot the XSF Content

In [ ]:
# modify to suppress/have UserWarnings and RuntimeWarnings
suppress_user_warnings = True
suppress_runtime_warnings = True
if suppress_user_warnings:
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)
if suppress_runtime_warnings:
    import warnings
    warnings.filterwarnings("ignore", category=RuntimeWarning)    

# modify to change the size of the plots
plt.rcParams['figure.dpi'] = 120

# modify to add/remove variables to ignore
vars_to_ignore = ["backscatter_r", ]

with nc.Dataset(xsf_path) as root:
    
    beam_group_ds = root['/root/sonar/beam/Beam_group1/']
    
    print(beam_group_ds.cmptypes)
    
    for var_name in beam_group_ds.variables:
        
        # ignore variables currently with issues
        if var_name in vars_to_ignore:
            print("SKIP: %s (listed to ignore)" % (var_name, )) 
            continue
               
        var = beam_group_ds.variables[var_name]       
        if var.ndim not in [1, 2]:
            print("SKIP: %s (unsupported nr of dims: %d)" % (var_name, var.ndim)) 
            continue
        if np.isnan(var).all():
            print("SKIP: %s (all nan values)" % (var_name, )) 
            continue            
        print("-> %s" % var_name)    
    
        plt.figure()
        
        try:
            plt.title("%s - min: %s, max: %s" % (var_name, np.nanmin(var), np.nanmax(var)))
        except:
            plt.title(var_name)
            
        try:
            if var.ndim == 1:
                plt.plot(var)
                
            elif var.ndim == 2:
                plt.imshow(var, origin='lower')

        except Exception as e:
            print("SKIP: %s (%s)" % (var_name, e))

The above plots show the content for the selected XSF file.